# Load data

In [ ]:
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/drive')
    path_to_file = '/content/drive/MyDrive/DSA4212_Assignment2/datasets'
    print(path_to_file)
    # move to Google Drive directory
    os.chdir(path_to_file)
    !pwd

Mounted at /content/drive
/content/drive/MyDrive/DSA4212_Assignment2/datasets
/content/drive/.shortcut-targets-by-id/1sKnwfHp4YEyYFgL7Arxyl1vY9sS5U5dH/DSA4212_Assignment2/datasets


In [ ]:
from sklearn.datasets import dump_svmlight_file
import numpy as np
import pandas as pd
import os
import urllib
import zipfile
from sklearn.model_selection import train_test_split
import shutil
 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam

%matplotlib inline

ratings = pd.read_table("ratings.dat",sep = '::',header=None,engine='python',names=['user_id','movie_id',"rating","timestamp"])
users = pd.read_table("users.dat",sep = '::',header=None,engine='python',names =['user_id','gender','age','occupation','zipcode'])
movies = pd.read_table('movies.dat', sep='::', header=None, engine='python', encoding='ISO-8859-1').to_numpy()


#movies.columns = ['movie_id','title','genres']

In [ ]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
avg_ratings=ratings.groupby(['movie_id']).mean()
count_rating=ratings.groupby(['movie_id']).size()

In [ ]:
avg_ratings['count']=count_rating

In [ ]:
avg_ratings.drop(['timestamp','user_id'], axis=1,inplace=True)

In [ ]:
users.head()

,user_id,gender,age,occupation,zipcode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [ ]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     6040 non-null   int64 
 1   gender      6040 non-null   object
 2   age         6040 non-null   int64 
 3   occupation  6040 non-null   int64 
 4   zipcode     6040 non-null   object
dtypes: int64(3), object(2)
memory usage: 236.1+ KB


In [ ]:
movies

array([[1, 'Toy Story (1995)', "Animation|Children's|Comedy"],
       [2, 'Jumanji (1995)', "Adventure|Children's|Fantasy"],
       [3, 'Grumpier Old Men (1995)', 'Comedy|Romance'],
       ...,
       [3950, 'Tigerland (2000)', 'Drama'],
       [3951, 'Two Family House (2000)', 'Drama'],
       [3952, 'Contender, The (2000)', 'Drama|Thriller']], dtype=object)

In [ ]:
for i in movies:
  i[2]= i[2].split('|')

In [ ]:
movies

array([[1, 'Toy Story (1995)',
        list(['Animation', "Children's", 'Comedy'])],
       [2, 'Jumanji (1995)',
        list(['Adventure', "Children's", 'Fantasy'])],
       [3, 'Grumpier Old Men (1995)', list(['Comedy', 'Romance'])],
       ...,
       [3950, 'Tigerland (2000)', list(['Drama'])],
       [3951, 'Two Family House (2000)', list(['Drama'])],
       [3952, 'Contender, The (2000)', list(['Drama', 'Thriller'])]],
      dtype=object)

In [ ]:
dfs = []
for i in movies:
  movie=[]
  movie.append(i[0])
  movie.append(i[1])
  movie.append(i[2])
  dfs.append(movie)

In [ ]:
movies = pd.DataFrame(dfs, columns=['movie_id','movie_title','genre'])


In [ ]:
movies

,movie_id,movie_title,genre
0,1,Toy Story (1995),"[Animation, Children's, Comedy]"
1,2,Jumanji (1995),"[Adventure, Children's, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama]"
4,5,Father of the Bride Part II (1995),[Comedy]
...,...,...,...
3878,3948,Meet the Parents (2000),[Comedy]
3879,3949,Requiem for a Dream (2000),[Drama]
3880,3950,Tigerland (2000),[Drama]
3881,3951,Two Family House (2000),[Drama]


In [ ]:
movies["year"] = movies["movie_title"].str[-5:-1]
movies["movie_title"] = movies["movie_title"].str[:-7]

In [ ]:
movies

,movie_id,movie_title,genre,year
0,1,Toy Story,"[Animation, Children's, Comedy]",1995
1,2,Jumanji,"[Adventure, Children's, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama]",1995
4,5,Father of the Bride Part II,[Comedy],1995
...,...,...,...,...
3878,3948,Meet the Parents,[Comedy],2000
3879,3949,Requiem for a Dream,[Drama],2000
3880,3950,Tigerland,[Drama],2000
3881,3951,Two Family House,[Drama],2000


In [ ]:
avg_ratings

,rating,count
movie_id,,
1,4.146846,2077
2,3.201141,701
3,3.016736,478
4,2.729412,170
5,3.006757,296
...,...,...
3948,3.635731,862
3949,4.115132,304
3950,3.666667,54


In [ ]:
movies=pd.merge(movies,avg_ratings,left_on='movie_id',right_index=True)
  

In [ ]:
movies

,movie_id,movie_title,genre,year,rating,count
0,1,Toy Story,"[Animation, Children's, Comedy]",1995,4.146846,2077
1,2,Jumanji,"[Adventure, Children's, Fantasy]",1995,3.201141,701
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,3.016736,478
3,4,Waiting to Exhale,"[Comedy, Drama]",1995,2.729412,170
4,5,Father of the Bride Part II,[Comedy],1995,3.006757,296
...,...,...,...,...,...,...
3878,3948,Meet the Parents,[Comedy],2000,3.635731,862
3879,3949,Requiem for a Dream,[Drama],2000,4.115132,304
3880,3950,Tigerland,[Drama],2000,3.666667,54
3881,3951,Two Family House,[Drama],2000,3.900000,40


In [ ]:
movies.rating=str(movies.rating)

In [ ]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [ ]:
movies['genre']=movies['genre'].apply(clean_data)

In [ ]:
def join(x):
    return ' '.join(x['genre']) + x['year'] + x['rating'] + str(x['count'])
movies['gen_yr'] = movies.apply(join, axis=1)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')

In [ ]:
count_matrix = count.fit_transform(movies['gen_yr'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
movies = movies.reset_index()
indices = pd.Series(movies.index, index=movies['movie_title'])

In [ ]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(movie_title, cosine_sim2=cosine_sim2):
    # Get the index of the movie that matches the title
    idx = indices[movie_title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim2[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return movies['movie_title'].iloc[movie_indices]

In [ ]:
get_recommendations('Toy Story', cosine_sim2)

52                      Big Green, The
12                               Balto
237                   Gumby: The Movie
304                 Swan Princess, The
354                     Lion King, The
580    Snow White and the Seven Dwarfs
581               Beauty and the Beast
582                          Pinocchio
601                    Aristocats, The
612            All Dogs Go to Heaven 2
Name: movie_title, dtype: object